 In this first step, we import necessary libraries and define key file paths and varaibles

In [6]:

import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


DATA_PATH = 'draft_data.csv'
SHAPEFILE_PATH = r'tl_2018_us_county.shp'
K = 5

#Define the tuition-related variables
tuition_cols = [
    'TU_costt4a', 'TU_npt4_priv', 'TU_npt4_pub', 'TU_pct_pell',
    'TU_tuitfte', 'TU_tuition_in', 'TU_tuition_out', 'TU_tuition_prog'
]


Load the monthly dataset and prepare it for annual aggregation by creating a datetime object and averaging. 

In [7]:

df_monthly = pd.read_csv(DATA_PATH)

df_monthly['month'] = pd.to_datetime(df_monthly['month'], errors='coerce').dt.month
df_monthly['year'] = df_monthly['year'].astype(int)
df_monthly['Date'] = pd.to_datetime(dict(year=df_monthly['year'], month=df_monthly['month'], day=1))

group_cols = ['FIPS', 'year']
df_yearly = df_monthly.groupby(group_cols)[tuition_cols].mean().reset_index()



#print the length of the original dataframe
print("Number of rows in monthly dataset:",len(df_monthly))

#Count rows where **all** tuition variables are NA
count_all_na_month= df_monthly[tuition_cols].isna().all(axis=1).sum()


#Count rows where **all** tuition variables are populated (no NAs)
count_all_populated_month = df_monthly[tuition_cols].notna().all(axis=1).sum()

#Missingness per tuition variable (monthly dataset)
missing_counts = df_monthly[tuition_cols].isna().sum()
print("Missingness per tuition variable (monthly dataset):")
print(missing_counts)

print("Rows with all tuition variables NA:", count_all_na_month)

print("Rows with all tuition variables populated:", count_all_populated_month)


#print the length of the yearly dataframe
print("Number of rows in yearly dataset:",len(df_yearly))

#Count rows where **all** tuition variables are NA
count_all_na = df_yearly[tuition_cols].isna().all(axis=1).sum()


#Count rows where **all** tuition variables are populated (no NAs)
count_all_populated = df_yearly[tuition_cols].notna().all(axis=1).sum()


print("Rows with all tuition variables NA:", count_all_na)

print("Rows with all tuition variables populated:", count_all_populated)



Number of rows in monthly dataset: 226152
Missingness per tuition variable (monthly dataset):
TU_costt4a         149868
TU_npt4_priv       173988
TU_npt4_pub        158700
TU_pct_pell        141480
TU_tuitfte         140988
TU_tuition_in      146904
TU_tuition_out     147504
TU_tuition_prog    182172
dtype: int64
Rows with all tuition variables NA: 138720
Rows with all tuition variables populated: 30588
Number of rows in yearly dataset: 18846
Rows with all tuition variables NA: 11560
Rows with all tuition variables populated: 2549


Loading shapefile and computing centroids

In [8]:

gdf = gpd.read_file(SHAPEFILE_PATH)
gdf = gdf.set_crs(epsg=4269).to_crs(epsg=5070)
gdf['centroid'] = gdf.geometry.centroid
gdf['centroid_x'] = gdf.centroid.x
gdf['centroid_y'] = gdf.centroid.y
gdf['GEOID'] = gdf['GEOID'].astype(str)

df_yearly['FIPS'] = df_yearly['FIPS'].astype(str).str.zfill(5)
df_yearly = df_yearly.merge(
    gdf[['GEOID', 'centroid_x', 'centroid_y']],
    left_on='FIPS',
    right_on='GEOID',
    how='left'
)

df_yearly[['FIPS', 'centroid_x', 'centroid_y']].head()

df_yearly_kfold=df_yearly

DataSourceError: tl_2018_us_county.shp: No such file or directory

Defining functions for imputation

In [ ]:

def prepare_features(df, target_col):
    mask = (
        df[target_col].notna() &
        df['centroid_x'].notna() &
        df['centroid_y'].notna()
    )
    features = ['centroid_x', 'centroid_y']
    X = df.loc[mask, features].copy()
    X_raw = X.values  
    return X_raw, df.loc[mask].index, None  


def knn_geo_impute(df, col, k=5):
    df_copy = df.copy()
    X_train_scaled, train_indices, scaler = prepare_features(df_copy, col)
    y_train = df_copy.loc[train_indices, col].values
    assert not np.isnan(y_train).any(), f"Training data for {col} contains missing values!"

    if len(X_train_scaled) == 0:
        print(f" No training data available for {col}, skipping imputation.")
        return df_copy, 0

    nbrs = NearestNeighbors(n_neighbors=min(k, len(X_train_scaled))).fit(X_train_scaled)

    mask_test = (
        df_copy[col].isna() &
        df_copy['centroid_x'].notna() &
        df_copy['centroid_y'].notna()
    )
    X_test = df_copy.loc[mask_test, ['centroid_x', 'centroid_y']].copy()
    if X_test.empty:
        return df_copy, 0

    X_test_scaled = X_test.values

    could_not_impute = 0

    for idx, x in tqdm(zip(X_test.index, X_test_scaled), total=X_test.shape[0], desc=f"Imputing {col}", leave=False):
        distances, neighbor_ids = nbrs.kneighbors([x], n_neighbors=k)
        neighbor_vals = y_train[neighbor_ids[0]]
        valid_vals = neighbor_vals[~np.isnan(neighbor_vals)]
        if len(valid_vals) > 0:
            df_copy.at[idx, col] = valid_vals.mean()
        else:
            could_not_impute += 1

    return df_copy, could_not_impute


Apply KNN imputation to each tuition column

In [ ]:
imputed_flags = pd.DataFrame(index=df_yearly.index)
print("Starting geo-only imputation at county-year level...")

for col in tqdm(tuition_cols, desc="Processing tuition variables"):
    original_known = df_yearly[col].notna()
    train_mask = (
        original_known & 
        df_yearly['centroid_x'].notna() & 
        df_yearly['centroid_y'].notna()
    )
    if train_mask.sum() < K:
        print(f" Skipping {col}: not enough known data to find {K} neighbors.")
        imputed_flags[col] = False
        continue

    df_yearly, could_not_impute = knn_geo_impute(df_yearly, col, k=K)
    now_known = df_yearly[col].notna()
    was_imputed = now_known & (~original_known)
    imputed_flags[col] = was_imputed

    


Starting geo-only imputation at county-year level...


Processing tuition variables: 100%|██████████| 8/8 [00:40<00:00,  5.05s/it]


Merge imputed yearly data back into the monthly dataset and save both versions

In [ ]:

df_monthly['FIPS'] = df_monthly['FIPS'].astype(str).str.zfill(5)
df_imputed = df_monthly.merge(
    df_yearly[group_cols + tuition_cols],
    on=['FIPS', 'year'],
    suffixes=('', '_imputed')
)

for col in tuition_cols:
    df_imputed[col] = df_imputed[col].fillna(df_imputed[f'{col}_imputed'])
    df_imputed.drop(columns=[f'{col}_imputed'], inplace=True)




In [33]:
#export imputed data, one with original year data that was used in the imputation, and another with the deaggregated imputated dataset
df_yearly.to_csv("imputed_data_geo_county_year.csv", index=False)
df_imputed.to_csv("imputed_data_geo_county_month.csv", index=False)

After imputation, we display how many values remain missing for each tuition variable.


In [34]:
#summary and remaining missing values after imputation
print("\nAll imputation complete and monthly data reconstructed.")
print(f"{df_yearly.shape[0]} county-year rows | {df_imputed.shape[0]} county-month rows")

print("\nFinal Missing Value Report:")
missing_summary = df_yearly[tuition_cols].isna().sum().to_frame(name='Missing Count')
print(missing_summary)


All imputation complete and monthly data reconstructed.
18846 county-year rows | 226152 county-month rows

Final Missing Value Report:
                 Missing Count
TU_costt4a                  12
TU_npt4_priv                12
TU_npt4_pub                 12
TU_pct_pell                 12
TU_tuitfte                  12
TU_tuition_in               12
TU_tuition_out              12
TU_tuition_prog             12


In this cell, we summarize imputation coverage across states and years. 


In [35]:
#inspecting count of imputed values per state
state_fips_map = {
    '01': 'Alabama', '02': 'Alaska', '04': 'Arizona', '05': 'Arkansas', '06': 'California',
    '08': 'Colorado', '09': 'Connecticut', '10': 'Delaware', '11': 'D.C.', '12': 'Florida',
    '13': 'Georgia', '15': 'Hawaii', '16': 'Idaho', '17': 'Illinois', '18': 'Indiana',
    '19': 'Iowa', '20': 'Kansas', '21': 'Kentucky', '22': 'Louisiana', '23': 'Maine',
    '24': 'Maryland', '25': 'Massachusetts', '26': 'Michigan', '27': 'Minnesota',
    '28': 'Mississippi', '29': 'Missouri', '30': 'Montana', '31': 'Nebraska', '32': 'Nevada',
    '33': 'New Hampshire', '34': 'New Jersey', '35': 'New Mexico', '36': 'New York',
    '37': 'North Carolina', '38': 'North Dakota', '39': 'Ohio', '40': 'Oklahoma',
    '41': 'Oregon', '42': 'Pennsylvania', '44': 'Rhode Island', '45': 'South Carolina',
    '46': 'South Dakota', '47': 'Tennessee', '48': 'Texas', '49': 'Utah', '50': 'Vermont',
    '51': 'Virginia', '53': 'Washington', '54': 'West Virginia', '55': 'Wisconsin',
    '56': 'Wyoming'
}
df_yearly['STATEFP'] = df_yearly['FIPS'].str[:2]
df_yearly['State'] = df_yearly['STATEFP'].map(state_fips_map)
df_yearly['was_any_tuition_imputed'] = imputed_flags[tuition_cols].any(axis=1)

impute_counts = (
    df_yearly[df_yearly['was_any_tuition_imputed']]
    .groupby(['State', 'year'])['FIPS']
    .nunique()
    .reset_index(name='NumCountiesImputed')
    .sort_values(['year', 'NumCountiesImputed'], ascending=[False, False])
)

print("\n Number of Counties Imputed by State and Year:")
print(impute_counts)


 Number of Counties Imputed by State and Year:
             State  year  NumCountiesImputed
258          Texas  2023                 254
60         Georgia  2023                 159
276       Virginia  2023                 132
102       Kentucky  2023                 120
150       Missouri  2023                 115
..             ...   ...                 ...
121  Massachusetts  2018                   5
61          Hawaii  2018                   3
169  New Hampshire  2018                   3
229   Rhode Island  2018                   3
43        Delaware  2018                   1

[301 rows x 3 columns]


In [36]:
# Load original yearly data (before imputation)
#I saved df_yearly_kfold after the step where I merged the centroid coordinates in the very beginning, below I'm just resaving df_yearly to be the df_yearly that was saved at the step after merging the centroids
df_yearly=df_yearly_kfold
# Tuition and columns
tuition_cols = [
    'TU_costt4a', 'TU_npt4_priv', 'TU_npt4_pub', 'TU_pct_pell',
    'TU_tuitfte', 'TU_tuition_in', 'TU_tuition_out', 'TU_tuition_prog'
]

# Cross-validation settings
kf = KFold(n_splits=5, shuffle=True, random_state=42)
K = 5  # Only K=5
results_list = []

print(f"\nRunning cross-validation for K = {K}")
for col in tuition_cols:
    valid_mask = (
        df_yearly[col].notna() &
        df_yearly['centroid_x'].notna() &
        df_yearly['centroid_y'].notna() 
    )
    df_valid = df_yearly.loc[valid_mask].reset_index(drop=True)
    if len(df_valid) < K:
        print(f" Skipping {col}: insufficient data for K = {K}")
        continue

    errors = []
    for train_idx, test_idx in kf.split(df_valid):
        train_df = df_valid.loc[train_idx]
        test_df = df_valid.loc[test_idx]

        train_X = train_df[['centroid_x', 'centroid_y']].values
        train_y = train_df[col].values

        test_preds = []
        test_true = []

        nbrs = NearestNeighbors(n_neighbors=min(K, len(train_X))).fit(train_X)

        for _, row in test_df.iterrows():
            x_test = row[['centroid_x', 'centroid_y']].values
            x_test = [x_test]  # keep as 2D array
            distances, neighbor_ids = nbrs.kneighbors(x_test, n_neighbors=K)
            neighbor_vals = train_y[neighbor_ids[0]]
            valid_vals = neighbor_vals[~np.isnan(neighbor_vals)]
            if len(valid_vals) > 0:
                pred = valid_vals.mean()
                test_preds.append(pred)
                test_true.append(row[col])

        if test_preds:
            mse = mean_squared_error(test_true, test_preds)
            rmse = np.sqrt(mse)
            errors.append(rmse)

    if errors:
        avg_rmse = np.mean(errors)
        results_list.append({'K': f"K = {K}", 'Variable': col, 'RMSE': avg_rmse})
        print(f"{col} — RMSE: {avg_rmse:.3f}")
    else:
        print(f"{col} — no valid folds to compute RMSE")

# Convert results to a DataFrame
results_df = pd.DataFrame(results_list)

# Compute mean and std dev for each tuition column (before imputation)
summary_stats = df_yearly[tuition_cols].agg(['mean', 'std']).T.reset_index()
summary_stats.columns = ['Variable', 'Mean', 'StdDev']

# Pivot RMSE results to get RMSE column for K=5
rmse_table = (
    results_df
    .pivot(index='Variable', columns='K', values='RMSE')
    .reset_index()
)

# Merge RMSE with summary stats
final_table = pd.merge(rmse_table, summary_stats, on='Variable', how='left')

# Round for display
final_table = final_table.round(3)

# Display the final table
print("\n Cross-validated RMSEs with Tuition Mean and Std Dev (K = 5):")
print(final_table.to_string(index=False))


Running cross-validation for K = 5
TU_costt4a — RMSE: 8256.277
TU_npt4_priv — RMSE: 4749.884
TU_npt4_pub — RMSE: 3055.235
TU_pct_pell — RMSE: 0.120
TU_tuitfte — RMSE: 9835.115
TU_tuition_in — RMSE: 6907.523
TU_tuition_out — RMSE: 6681.259
TU_tuition_prog — RMSE: 5059.953

 Cross-validated RMSEs with Tuition Mean and Std Dev (K = 5):
       Variable    K = 5      Mean    StdDev
     TU_costt4a 8256.277 25141.946 15460.875
   TU_npt4_priv 4749.884 19038.597  7222.179
    TU_npt4_pub 3055.235  9812.648  5027.152
    TU_pct_pell    0.120     0.394     0.181
     TU_tuitfte 9835.115  8324.475 12274.252
  TU_tuition_in 6907.523 13839.635 13590.106
 TU_tuition_out 6681.259 17775.735 12590.771
TU_tuition_prog 5059.953 14598.132  8601.613
